# Display of the mission parameters Kodiak100

In [ ]:
import os.path as pth
import logging

import fastoad.api as oad
import fastga_he.api as oad_he
import fastga.utils.postprocessing.post_processing_api as api_plots

DATA_FOLDER_PATH = "data"
RESULTS_FOLDER_PATH = "results"
WORK_FOLDER_PATH = "workdir"

CONFIGURATION_FILE_REF = pth.join(DATA_FOLDER_PATH, "full_sizing_kodiak100.yml")
PT_FILE_REF = pth.join(DATA_FOLDER_PATH, "turboshaft_propulsion.yml")
OUTPUT_FILE_REF = pth.join(RESULTS_FOLDER_PATH, "oad_process_outputs_ref.xml")
MISSION_FILE_REF = pth.join(RESULTS_FOLDER_PATH, "turboshaft_propulsion.csv")
PT_WATCHER_FILE_REF = pth.join(RESULTS_FOLDER_PATH, "fuel_propulsion_pt_watcher.csv")

CONFIGURATION_FILE_HYB = pth.join(DATA_FOLDER_PATH, "full_sizing_hybrid_kodiak.yml")
PT_FILE_HYB = pth.join(DATA_FOLDER_PATH, "hybrid_propulsion_full_sizing.yml")
OUTPUT_FILE_HYB = pth.join(RESULTS_FOLDER_PATH, "oad_process_outputs_full_sizing_hybrid.xml")
MISSION_FILE_HYB = pth.join(RESULTS_FOLDER_PATH, "full_sizing_hybrid_propulsion.csv")
PT_WATCHER_FILE_HYB = pth.join(RESULTS_FOLDER_PATH, "hybridized_full_sizing_kodiak.csv")

# For having log messages on screen
logging.basicConfig(level=logging.WARNING, format="%(levelname)-8s: %(message)s")

# Preprocessing

Because the post processing function expects SI units and don't really check, there can be some weird graphs if some data are not post processed. This next cell is meant to solve that

In [ ]:
from openmdao.utils.units import convert_units

datafile_hyb = oad.DataFile(OUTPUT_FILE_HYB)

# Convert fuselage length to m
fuselage_length = convert_units(
    datafile_hyb["data:geometry:fuselage:length"].value[0],
    datafile_hyb["data:geometry:fuselage:length"].units,
    "m",
)
datafile_hyb["data:geometry:fuselage:length"].value = fuselage_length
datafile_hyb["data:geometry:fuselage:length"].units = "m"
datafile_hyb.save()

datafile_ref = oad.DataFile(OUTPUT_FILE_REF)

# Convert fuselage length to m
fuselage_length = convert_units(
    datafile_ref["data:geometry:fuselage:length"].value[0],
    datafile_ref["data:geometry:fuselage:length"].units,
    "m",
)
datafile_ref["data:geometry:fuselage:length"].value = fuselage_length
datafile_ref["data:geometry:fuselage:length"].units = "m"
datafile_ref.save()

In [ ]:
from IPython.display import IFrame

NETWORK_FILE_REF = pth.join(WORK_FOLDER_PATH, "kodiak_assembly_ref.html")

oad_he.power_train_network_viewer(
    power_train_file_path=PT_FILE_REF, network_file_path=NETWORK_FILE_REF
)

# For some reason, this doesn't display icon. Opening it in Firefox does the trick.
IFrame(src=NETWORK_FILE_REF, width="100%", height="500px")

In [ ]:
from IPython.display import IFrame

NETWORK_FILE_HYB = pth.join(WORK_FOLDER_PATH, "kodiak_assembly_hyb.html")

oad_he.power_train_network_viewer(
    power_train_file_path=PT_FILE_HYB, network_file_path=NETWORK_FILE_HYB
)

# For some reason, this doesn't display icon. Opening it in Firefox does the trick.
IFrame(src=NETWORK_FILE_HYB, width="100%", height="500px")

In [ ]:
fig = api_plots.aircraft_geometry_plot(OUTPUT_FILE_REF, name="Reference Kodiak 100")
fig = api_plots.aircraft_geometry_plot(OUTPUT_FILE_HYB, name="Hybrid Kodiak 100", fig=fig)
fig.update_layout(height=800, width=1200)
fig.show()

In [ ]:
fig = api_plots.mass_breakdown_bar_plot(OUTPUT_FILE_REF, name="Reference Kodiak 100")
fig = api_plots.mass_breakdown_bar_plot(OUTPUT_FILE_HYB, name="Hybrid Kodiak 100", fig=fig)
fig.update_layout(height=800, width=1200)
fig.show()

In [ ]:
mission = oad.MissionViewer()
mission.add_mission(MISSION_FILE_REF, name="Reference Kodiak 100")
mission.add_mission(MISSION_FILE_HYB, name="Hybrid Kodiak 100")

mission.display()

# Electric Kodiak 100

In [ ]:
import pathlib 
from fastga_he.gui.performances_viewer import PerformancesViewer

DATA_FOLDER_PATH = "data"
RESULTS_FOLDER_PATH = "results"
WORK_FOLDER_PATH = "workdir"

MISSION_DATA_FILE = pathlib.Path(RESULTS_FOLDER_PATH) / "elec_kodiak_propulsion.csv"
PT_DATA_FILE = pathlib.Path(RESULTS_FOLDER_PATH) / "elec_kodiak_power_train_data.csv"

perfo_viewer = PerformancesViewer(
    power_train_data_file_path=PT_DATA_FILE.as_posix(),
    mission_data_file_path=MISSION_DATA_FILE.as_posix(),
    plot_height=800,
)